Compute a flag based on seed clustereing

In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Set path to data folder
current_dir = Path(os.getcwd())
DATA_DIR = os.path.join(current_dir.parent.parent, 'data-regen-rangers')
DATA_DIR_GITCOIN = os.path.join(current_dir.parent.parent, 'data-gitcoin')
GRANT_ROUND_ID = '0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc'
ODC_DATA_DIR = os.path.join(current_dir.parent.parent.parent, 'data')

PATH_TO_VOTES = os.path.join(DATA_DIR_GITCOIN, 'citizen-votes.csv')
# PATH_TO_GRANTS = os.path.join(DATA_DIR, "all-allo-rounds.csv")
# PATH_TO_PROJECTS = os.path.join(DATA_DIR, "projects_QmQurt.csv")

In [4]:
df_matching_address = pd.read_csv(f'../output_gitcoin/full_features/voters_features_{GRANT_ROUND_ID}.csv')
df_votes = pd.read_csv(f'../output_gitcoin/full_features/votes_features_citizen_last_{GRANT_ROUND_ID}.csv')
df_seed = pd.read_csv(os.path.join(DATA_DIR, 'seed_wallet_citizen.csv'))

In [5]:
df_matching_address.columns

Index(['address', 'seed_same_naive', 'seed_same', 'seed_suspicious',
       'less_5_tx', 'less_10_tx', 'interacted_other_ctbt', 'lcs',
       'cluster_size_lcs', 'mean_score_lcs', 'max_score_lcs', 'has_lcs',
       'count_interaction_with_pool', 'count_interaction_with_toxic',
       'interact_less_5tx', 'has_interaction_toxic', 'has_no_pool_interaction',
       'count_interaction_with_airdrop_m', 'is_airdrop_master',
       'count_interaction_with_tornado', 'count_interaction_with_disperse',
       'has_interaction_airdrop_m', 'has_interaction_tornado',
       'has_interaction_disperse', 'flagged'],
      dtype='object')

In [6]:
df_votes.columns

Index(['block_timestamp', 'tx_hash', 'voter', 'project', 'amount_usd',
       '__row_index', 'seed_same_naive', 'seed_same', 'seed_suspicious',
       'less_5_tx', 'less_10_tx', 'interacted_other_ctbt', 'lcs',
       'cluster_size_lcs', 'mean_score_lcs', 'max_score_lcs', 'has_lcs',
       'count_interaction_with_pool', 'count_interaction_with_toxic',
       'interact_less_5tx', 'has_interaction_toxic', 'has_no_pool_interaction',
       'count_interaction_with_airdrop_m', 'is_airdrop_master',
       'count_interaction_with_tornado', 'count_interaction_with_disperse',
       'has_interaction_airdrop_m', 'has_interaction_tornado',
       'has_interaction_disperse', 'flagged', 'stakeridoo_detected',
       'doge_detected', 'odc_detected', 'has_seed_cluster', 'cluster_number',
       'really_suspicicious_cluster'],
      dtype='object')

In [7]:
df_seed.head(2)

,EOA,from_address,to_address
0,0x000000006f457c0f8f560333d9c2877287d92a92,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x000000006f457c0f8f560333d9c2877287d92a92
1,0x000128fa45d79dc9af8016da242781f12c363fd5,0xe4edb277e41dc89ab076a1f049f4a3efa700bce8,0x000128fa45d79dc9af8016da242781f12c363fd5


There is no duplicate votes in the votes data frame

In [16]:
print(f'Number of votes: {df_votes.shape[0]}')
print(f'Number of unique votes: {df_votes.drop_duplicates().shape[0]}')
print(f'Number of unique voters: {df_votes.voter.nunique()}')

Number of votes: 57366
Number of unique votes: 57366
Number of unique voters: 17023


In [19]:
df_votes.groupby('voter').count().tx_hash.describe()

count    17023.000000
mean         3.369911
std          4.748125
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         99.000000
Name: tx_hash, dtype: float64

In [68]:
flagging_legos = ['has_lcs', 'has_interaction_toxic', 'has_interaction_airdrop_m', 'has_interaction_disperse', 'is_airdrop_master', 'interact_less_5tx', 'flagged', 'stakeridoo_detected', 'doge_detected', 'really_suspicicious_cluster', 'odc_detected']

In [69]:
df_votes[flagging_legos].sum()

has_lcs                         3223
has_interaction_toxic              0
has_interaction_airdrop_m       2606
has_interaction_disperse         128
is_airdrop_master               3463
interact_less_5tx                101
flagged                         8626
stakeridoo_detected              983
doge_detected                      6
really_suspicicious_cluster     2814
odc_detected                   11430
dtype: int64

In [76]:
columns_expected_matching = np.append(df_matching_address.columns.values, ['stakeridoo_detected', 'doge_detected', 'really_suspicicious_cluster', 'odc_detected'])
not_address = [True if x != 'address' else False for x in columns_expected_matching]
columns_expected_matching = columns_expected_matching[not_address]
columns_expected_matching = np.append(columns_expected_matching, ['voter'])

In [77]:
df_matching = df_votes.drop_duplicates('voter').loc[:, columns_expected_matching]
df_matching.rename(axis='columns', mapper={'voter': 'address'}, inplace=True)

In [78]:
df_matching.to_csv(f'../output_gitcoin/full_features/voters_features_{GRANT_ROUND_ID}.csv', index=False)

In [79]:
df_matching[flagging_legos].sum()

has_lcs                        1082
has_interaction_toxic             0
has_interaction_airdrop_m       591
has_interaction_disperse         37
is_airdrop_master               728
interact_less_5tx                34
flagged                        2282
stakeridoo_detected             136
doge_detected                     2
really_suspicicious_cluster     758
odc_detected                   2976
dtype: int64